Imports

In [ ]:
import sys
sys.path.append("/home/sanketh/DANI/GraphMatching/")
import torch.multiprocessing as mp
mp.set_start_method("spawn")

In [ ]:
import pathlib
import os
import numpy as np
import torch
from torch import optim
from subgraph_matching_via_nn.data.data_loaders import load_graph
from powerful_gnns.util import S2VGraph
from powerful_gnns.util import load_data_given_graph_list_and_label_map
from powerful_gnns.models.graphcnn import GraphCNN
from powerful_gnns.util import separate_data
from powerful_gnns.classifier_training import train, test
from common.EmbeddingCalculationsService import pairwise_l2_distance, show_distance_matrix, \
    calculate_energy_based_hidden_rep
from subgraph_matching_via_nn.training.PairSampleInfo import Pair_Sample_Info
from subgraph_matching_via_nn.data.annotated_graph import AnnotatedGraph
from subgraph_matching_via_nn.graph_metric_networks.graph_metric_nn import MLPGraphMetricNetwork
from subgraph_matching_via_nn.graph_metric_networks.embedding_metric_nn import EmbeddingMetricNetwork
from subgraph_matching_via_nn.graph_embedding_networks.gnn_embedding_network import GNNEmbeddingNetwork
from subgraph_matching_via_nn.training.trainer.S2VGraphEmbeddingSimilarityMetricTrainer import \
    S2VGraphEmbeddingSimilarityMetricTrainer
from subgraph_matching_via_nn.graph_metric_networks.graph_metric_nn import SingleEmbeddingGraphMetricNetwork
from subgraph_matching_via_nn.graph_metric_networks.graph_metric_nn import S2VGraphEmbeddingGraphMetricNetwork
from networkx import NetworkXError
import itertools
import networkx as nx
import contextlib
import joblib
from tqdm import tqdm
from joblib import Parallel, delayed
from os import cpu_count
import pickle
from common.graph_utils import GED_graph_generator

In [ ]:
@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

In [ ]:
def generate_s2v_graphs(networkx_graphs, print_stats=True):
    g_list = []
    label_dict = {}
    for networkx_graph in networkx_graphs:
        g_list.append(S2VGraph(networkx_graph, label=None))

        if not label in label_dict:
            mapped = len(label_dict)
            label_dict[label] = mapped

    graphs, _ = load_data_given_graph_list_and_label_map(g_list, label_dict, degree_as_tag=True, device=device, print_stats=print_stats)
    for s2v_graph in graphs:
        # convert graph features here, not only in trainer! (heatmap is probably wrong)
        annotated_graph = AnnotatedGraph(s2v_graph.g)
        s2v_graph.node_features = annotated_graph.node_indicator.to(device=device)

    return graphs

In [ ]:
goal_subcircuit_name = 'alu'
DATA_PATH = 'C:/Users/kogan/OneDrive/Desktop/Research/AMIT/GraphMatching/subgraph_matching_via_nn/data/subcircuits/'
DATA_PATH = '../subgraph_matching_via_nn/data/subcircuits/'

desktop = pathlib.Path(DATA_PATH)
subgraphs = []
labels = []
for circuit_dir in desktop.iterdir():
    if circuit_dir.is_dir():
        for subcircuit_file in circuit_dir.iterdir():
            if subcircuit_file.is_file():
                file_name = subcircuit_file.name
                if file_name == 'full_graph.p':
                    file_rel_path = f"{os.sep}{file_name}"
                    loader_params = {
                     'data_path' : str(circuit_dir),
                     'g_full_path': file_rel_path,
                     'g_sub_path': file_rel_path}

                    sub_graph = \
                        load_graph(type='subcircuit',
                                   loader_params=loader_params)

                    if goal_subcircuit_name in circuit_dir.name:
                        labels.append(1)
                    else:
                        # continue #TODO?
                        labels.append(0)
                    subgraphs.append(sub_graph)

labels = np.array(labels, dtype='float32')
N = len(subgraphs)
N_training = int(2 / 3 * N)

In [ ]:
subgraphs = []
labels = []

circuit_base_dir = 'C:\\Users\\kogan\\OneDrive\\Desktop\\Research\\AMIT\\GraphMatching\\subgraph_matching_via_nn\\data\\subcircuits\\'
circuit_base_dir = '/home/sanketh/DANI/GraphMatching/subgraph_matching_via_nn/data/subcircuits/'


for circuit_file_name in ['adder_4', 'alu_4', 'alu_8', 'alu_16', 'alu_32', 'mul_4_4', 'mul_4_8', 'mul_8_8', 'mul_16_16', 'mul_16_32']:
    file_rel_path = 'full_graph.p'
    circuit_dir = f"{circuit_base_dir}{circuit_file_name}{os.sep}"
    loader_params = {
     'data_path' : str(circuit_dir),
     'g_full_path': file_rel_path,
     'g_sub_path': file_rel_path}

    sub_graph = \
        load_graph(type='subcircuit',
                   loader_params=loader_params)
    subgraphs.append(sub_graph)
    labels.append(0)
labels[0] = 1

N = len(subgraphs)
N_training = int(2 / 3 * N)

Transform graphs into S2VGraph-s

In [ ]:
g_list = []
label_dict = {}
for subgraph, label in zip(subgraphs, labels):
    g_list.append(S2VGraph(subgraph.G, label))

    if not label in label_dict:
        mapped = len(label_dict)
        label_dict[label] = mapped

Process graph features

In [ ]:
device = 'cpu' if torch.cuda.is_available() else 'cpu'

graphs, num_classes = load_data_given_graph_list_and_label_map(g_list, label_dict, degree_as_tag=True, device=device)

Setup model and training

In [ ]:
#set up seeds and gpu device
torch.manual_seed(0)
np.random.seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(0)

##k-fold cross validation. Conduct an experiment on the fold specified by args.fold_idx.
# train_graphs, test_graphs = separate_data(graphs, seed=0, fold_idx=0, n_splits=1)
# train_graphs = graphs[:4]
# test_graphs = graphs[4:]
train_graphs = test_graphs = graphs

model = GraphCNN(num_layers=5, num_mlp_layers = 2, input_dim=train_graphs[0].node_features.shape[1], hidden_dim=64, output_dim=num_classes, final_dropout=0.5, learn_eps=False, graph_pooling_type="sum", neighbor_pooling_type="sum", device=device)

optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

Train and Test

In [ ]:
from livelossplot import PlotLosses

liveloss = PlotLosses(mode='notebook')

filename = ""
epochs = 3
k_update_plot = 25

for epoch in range(1, epochs + 1):
    scheduler.step()

    avg_loss = train(iters_per_epoch=50, batch_size=len(train_graphs), model=model, device=device, train_graphs=train_graphs, optimizer=optimizer, epoch=epoch)
    acc_train, acc_test = test(model, device, train_graphs, test_graphs, epoch)

    if not filename == "":
        with open(filename, 'w') as f:
            f.write("%f %f %f" % (avg_loss, acc_train, acc_test))
            f.write("\n")
    print("")

    print(model.eps)

    if epoch % k_update_plot == 0:
        liveloss.update({'train error': avg_loss.item()})
        liveloss.send()

Show predictions

In [ ]:
model.eval()
pred = model(train_graphs).max(1, keepdim=True)[1]
print(pred)

In [ ]:
pred = model(test_graphs).max(1, keepdim=True)[1]
print(pred)

show all distances matrix (margin)

In [ ]:
all_embeddings = model.get_embedding(graphs)

In [ ]:
l2_dists = pairwise_l2_distance(all_embeddings)

rounding_constant = 10 ** 3
cos_dists = torch.round(calculate_energy_based_hidden_rep(all_embeddings, threshold=-1) * rounding_constant) / rounding_constant

show_distance_matrix(l2_dists, "l2-distances")
show_distance_matrix(cos_dists, "cosine-distances")

what is the smallest margin?

In [ ]:
def get_min_non_diagonal_entry(tensor_):
    tensor_with_inf_diag = tensor_ + torch.diag_embed(torch.ones(tensor_.shape[0], device=device) * float("inf"))
    return torch.min(tensor_with_inf_diag).item()

print(get_min_non_diagonal_entry(l2_dists))
print(get_min_non_diagonal_entry(cos_dists))

Contrastive loss training

Assemble graph pairs data

In [ ]:
train_samples_list = []
reference_subgraph_index = 0
reference_subgraph = subgraphs[reference_subgraph_index]
negative_examples_subgraphs = subgraphs[1:]

for subgraph in subgraphs:
    if subgraph == reference_subgraph:
        is_negative_example = False
    else:
        is_negative_example = True

    G1_annotated = AnnotatedGraph(subgraph.G)
    G2_annotated = AnnotatedGraph(reference_subgraph.G)
    train_samples_list.append(Pair_Sample_Info(
        subgraph=G1_annotated,
        masked_graph=G2_annotated,
        is_negative_sample=torch.tensor(is_negative_example)))

val_samples_list = train_samples_list

Define GNN model

In [ ]:
# input dim matches the node mask (w) shape
input_dim = S2VGraphEmbeddingSimilarityMetricTrainer.get_model_expected_input_dim(train_samples_list[0])
model_factory_func = lambda device: GraphCNN(num_layers=5, num_mlp_layers = 2, input_dim=input_dim, hidden_dim=64, output_dim=num_classes, final_dropout=0.5, learn_eps=False, graph_pooling_type="sum", neighbor_pooling_type="sum", device=device)

Define graph metric to train, and the trainer

In [ ]:
solver_params = {
    "device": device,
    "lr": 1e-3, "weight_decay": 1e-3,
    "max_epochs": 50,
    "cycle_patience": 5, "step_size_up": 10, "step_size_down": 10,
    "loss_convergence_threshold": None,
    "train_loss_convergence_threshold": 1e-3,
    "successive_convergence_min_iterations_amount": 5,
    "margin_loss_margin_value": 5,
    "max_grad_norm": 0.1,
    "k_update_plot": 1,
    "train_monitoring_epochs_pace": 5,
    "batch_size": len(subgraphs),
    "num_workers": 0,
    "is_use_model_compliation": False, #Only supported for Pytorch > 2.0
}

problem_params = {"input_dim": input_dim}

dump_base_path = f".{os.sep}runlogs"

def init_embedding_net_and_trainer(model_factory_func):

    # must start on CPU to allow moving model to GPU, due to existing pytorch bug
    device = 'cpu'
    model = model_factory_func(device=device)

    if solver_params['is_use_model_compliation']:
        model = torch.compile(model)

    loss_fun = torch.nn.MSELoss()
    embedding_metric_network = EmbeddingMetricNetwork(loss_fun=loss_fun)

    gnn_embedding_nn = GNNEmbeddingNetwork(gnn_model=model)

    # embedding_nns = \
    #     [
    #         GNNEmbeddingNetwork(gnn_model=model),
    #     ]

    graph_metric_nn = S2VGraphEmbeddingGraphMetricNetwork(embedding_network=gnn_embedding_nn,
                                           embdding_metric_network=embedding_metric_network,
                                           device=device)

    # graph_metric_nn = SingleEmbeddingGraphMetricNetwork(embedding_network=embedding_nns[0],
    #                                        embdding_metric_network=embedding_metric_network,
    #                                        device=device)

    # graph_metric_nn = MLPGraphMetricNetwork(embedding_networks=embedding_nns,
    #                                         embdding_metric_network=embedding_metric_network,
    #                                         device=device)


    trainer = S2VGraphEmbeddingSimilarityMetricTrainer(graph_metric_nn, dump_base_path,
                                      problem_params, solver_params)

    return trainer, graph_metric_nn, model

trainer, graph_metric_nn, model = init_embedding_net_and_trainer(model_factory_func)

Train

In [ ]:
def show_distances_heatmap(graphs, model):

    model.eval()
    all_embeddings = model.get_embedding(graphs)

    l2_dists = pairwise_l2_distance(all_embeddings)

    rounding_constant = 10 ** 3
    cos_dists = torch.round(calculate_energy_based_hidden_rep(all_embeddings, threshold=-1) * rounding_constant) / rounding_constant

    show_distance_matrix(l2_dists, "l2-distances")
    show_distance_matrix(cos_dists, "cosine-distances")
    print(f"Off matrix diagonal margin: {get_min_non_diagonal_entry(l2_dists)}")
    print(f"Off matrix diagonal margin: {get_min_non_diagonal_entry(cos_dists)}")

show_distances_heatmap(generate_s2v_graphs([graph.g for graph in graphs]), model)

In [ ]:
len(train_samples_list)

In [ ]:
trainer.train(processes_device_ids=[0], use_existing_data_loaders=False, train_samples_list=train_samples_list, val_samples_list=val_samples_list)

Add ged=1 examples as positive examples

In [ ]:
def generate_perturbed_graphs(reference_graph: nx.Graph):
    # assumption: there are no self loops, and no duplicate edges of opposite directions

    # graphs generated by removing an existing edge
    for edge in reference_graph.edges:
        copy_graph = reference_graph.copy()
        copy_graph.remove_edge(*edge)
        yield copy_graph

    # graphs generated by adding a missing edge
    sorted_nodes = sorted(reference_graph.nodes, reverse=False)
    for i, node_i in enumerate(sorted_nodes):
        if i == len(sorted_nodes) - 1:
            break

        for node_j in sorted_nodes[i+1:]:
            edge = (node_i, node_j)
            if reference_graph.has_edge(*edge):
                continue
            copy_graph = reference_graph.copy()
            copy_graph.add_edge(*edge)
            yield copy_graph

# generate subgraphs for which ged(subgraph, reference_subgraph)==1
perturbed_graphs = list(generate_perturbed_graphs(reference_subgraph.G_sub))
for perturbed_graph in perturbed_graphs:
    is_negative_example = False

    G1_annotated = AnnotatedGraph(perturbed_graph)
    G2_annotated = AnnotatedGraph(reference_subgraph.G)
    train_samples_list.append(Pair_Sample_Info(
        subgraph=G1_annotated,
        masked_graph=G2_annotated,
        is_negative_sample=torch.tensor(is_negative_example)))

In [ ]:
len(train_samples_list)

In [ ]:
val_samples_list = train_samples_list
model_factory_func = lambda device: GraphCNN(num_layers=5, num_mlp_layers = 2, input_dim=input_dim, hidden_dim=64, output_dim=num_classes, final_dropout=0.5, learn_eps=False, graph_pooling_type="sum", neighbor_pooling_type="sum", device=device)
solver_params["batch_size"] = len(train_samples_list)

trainer, graph_metric_nn, model = init_embedding_net_and_trainer(model_factory_func)

trainer.train(processes_device_ids=[0], use_existing_data_loaders=False, train_samples_list=train_samples_list, val_samples_list=val_samples_list)

In [ ]:
all_s2v_graphs = generate_s2v_graphs([subgraph.G for subgraph in subgraphs] + perturbed_graphs)
show_distances_heatmap(all_s2v_graphs, model)

In [ ]:
show_distances_heatmap(all_s2v_graphs, model)

In [ ]:
show_distances_heatmap(all_s2v_graphs[:len(subgraphs)], model)

In [ ]:
reference_s2v_graph = all_s2v_graphs[reference_subgraph_index]
reference_graph_with_positive_examples = [reference_s2v_graph] + all_s2v_graphs[len(subgraphs):]
reference_graph_with_negative_examples = [all_s2v_graphs[i] for i in range(len(all_s2v_graphs[:len(subgraphs)])) if i != reference_subgraph_index]

show_distances_heatmap(reference_graph_with_positive_examples, model)

In [ ]:
all_embeddings = model.get_embedding(reference_graph_with_positive_examples)
l2_dists = pairwise_l2_distance(all_embeddings)

In [ ]:
model.eval()

In [ ]:
positive_distances = []
for positive_example_second_graph in reference_graph_with_positive_examples:
    distance = graph_metric_nn.forward([(reference_s2v_graph, positive_example_second_graph)]).item()
    positive_distances.append(distance)
    if distance > solver_params['margin_loss_margin_value']:
        print(f"positive loss term not zero, as distance for this positive example pair is {distance}")

negative_distances = []
for negative_example_second_graph in reference_graph_with_negative_examples:
    distance = graph_metric_nn.forward([(reference_s2v_graph, negative_example_second_graph)]).item()
    negative_distances.append(distance)
    if distance < solver_params['margin_loss_margin_value']:
        print(f"negative loss term not zero, as distance for this negative example pair is {distance}")

In [ ]:
max(positive_distances)

In [ ]:
min(negative_distances)

k-subgraphs experiment

In [ ]:
def generate_pair_example(G1_annotated, G2_annotated, is_negative_example):
    return Pair_Sample_Info(
        subgraph=G1_annotated,
        masked_graph=G2_annotated,
        is_negative_sample=torch.tensor(is_negative_example))

def compare_graphs_and_generate_pair_example(G1_annotated, G2_annotated):
    SG1 = G1_annotated.g
    SG2 = G2_annotated.g

    if nx.is_isomorphic(SG1, SG2):
        isomorphic_pairs.append((SG1, SG2))
        return generate_pair_example(G1_annotated, G2_annotated, is_negative_example = False)
    else:
        return generate_pair_example(G1_annotated, G2_annotated, is_negative_example = True)
    
    # try:
    #     diff_graph = nx.symmetric_difference(SG1, SG2)
    # except NetworkXError:
    #     # node sets are different
    #     if nx.is_empty(SG1) and nx.is_empty(SG2) and (len(SG1) == len(SG2)):
    #         # positive example
    #         print("node sets are different, but isomorphic graphs")
    #         return generate_pair_example(G1_annotated, G2_annotated, is_negative_example = False)
    #     else:
    #         # negative example
    #         print("node sets are different, and not isomorphic graphs")
    #         return generate_pair_example(G1_annotated, G2_annotated, is_negative_example = True)

    # if nx.is_empty(diff_graph):
    #     # print("isomorphic graphs")
    #     isomorphic_pairs.append((SG1, SG2))
    #     # positive example
    #     return generate_pair_example(G1_annotated, G2_annotated, is_negative_example = False)
    # else:
    #     # negative example
    #     return generate_pair_example(G1_annotated, G2_annotated, is_negative_example = True)

K = 11
full_graph = reference_subgraph.G

subgraphs_iterator = itertools.combinations(full_graph, K)
k_subgraphs = [full_graph.subgraph(s) for s in subgraphs_iterator]
k_subgraph_annotated_graphs = [AnnotatedGraph(g) for g in k_subgraphs]

train_samples_list = []
isomorphic_pairs = []
train_sample_indices_tuple_list = []

In [ ]:
for subgraph_counter1, SG_annotated_1 in enumerate(k_subgraph_annotated_graphs):
    for subgraph_counter2 in range(subgraph_counter1+1, len(k_subgraph_annotated_graphs)):
        SG_annotated_2 = k_subgraph_annotated_graphs[subgraph_counter2]
        train_samples_list.append(compare_graphs_and_generate_pair_example(SG_annotated_1, SG_annotated_2))
        train_sample_indices_tuple_list.append((subgraph_counter1, subgraph_counter2))
        # if train_samples_list[-1].is_negative_sample == False:
        #     print(f"{subgraph_counter1} vs {subgraph_counter2}")

In [ ]:
# ged_pairs_dump_base_path = f".{os.sep}ged_pairs

# loaded_ged_examples = []
# for i, doc_path in enumerate(pathlib.Path(ged_pairs_dump_base_path).rglob("*.p*")):
#     k_subgraph_ged_pairs_list = pickle.load(open(doc_path, 'rb'))
#     loaded_ged_examples += k_subgraph_ged_pairs_list
#     print(i)

# if not os.path.exists(ged_pairs_dump_base_path):
#     os.makedirs(ged_pairs_dump_base_path)
#
# # k_subgraph_to_perturbed_graphs_map = {}
# def generate_ged_paris(subgraph_counter1):
#     train_samples_list = []
#     SG1 = k_subgraph_annotated_graphs[subgraph_counter1]
#     ged1_perturbed_graphs = list(generate_perturbed_graphs(SG1.g))
#     for ged1_perturbed_graph in ged1_perturbed_graphs:
#         # positive example
#         train_samples_list.append(generate_pair_example(SG1, AnnotatedGraph(ged1_perturbed_graph), is_negative_example = False))
#
#         ged2_perturbed_graphs = list(generate_perturbed_graphs(ged1_perturbed_graph))
#         for ged2_perturbed_graph in ged2_perturbed_graphs:
#             # could be positive or negative example
#             train_samples_list.append(compare_graphs_and_generate_pair_example(k_subgraph_annotated_graphs[subgraph_counter1], AnnotatedGraph(ged2_perturbed_graph)))
#
#     path = os.path.join(ged_pairs_dump_base_path, f"subgraph_{subgraph_counter1}.p")
#     with open(path, 'wb') as f:
#         pickle.dump(train_samples_list, f)
#     return train_samples_list
#
# total_tasks = len(k_subgraphs)
# with tqdm_joblib(tqdm(desc="My calculation", total=total_tasks)) as progress_bar:
#     #Parallel(n_jobs=16)(delayed(long_task)() for i in range(10))
#
#     ged_examples = Parallel(n_jobs=int(cpu_count()), prefer='processes')(
#         delayed(generate_ged_paris)(subgraph_counter1=subgraph_counter1)
#         for subgraph_counter1 in range(total_tasks)
#     )
#
# # generate subgraphs for which GED==1
# for subgraph_counter1, SG1 in enumerate(k_subgraphs):
#     print(subgraph_counter1)
#     ged1_perturbed_graphs = list(generate_perturbed_graphs(SG1))
#     for ged1_perturbed_graph in ged1_perturbed_graphs:
#         # positive example
#         train_samples_list.append(generate_pair_example(SG1, ged1_perturbed_graph, is_negative_example = False))
#         #
#         # ged2_perturbed_graphs = list(generate_perturbed_graphs(ged1_perturbed_graph))
#         # for ged2_perturbed_graph in ged2_perturbed_graphs:
#         #     # could be positive or negative example
#         #     train_samples_list.append(compare_graphs_and_generate_pair_example(k_subgraph_annotated_graphs[subgraph_counter1], AnnotatedGraph(ged2_perturbed_graph)))
#     # k_subgraph_to_perturbed_graphs_map[SG1] = perturbed_graphs

#
#
# # generate all pairs examples
# for SG1, SG1_perturbed_graphs in k_subgraph_to_perturbed_graphs_map.items():
#     for SG2, SG2_perturbed_graphs in k_subgraph_to_perturbed_graphs_map.items():
#
#         if SG1 == SG2:
#             continue # trivial pairs, or pairs already considered
#
#         for SG2_perturbed_graph in SG2_perturbed_graphs:
#             train_samples_list.append(compare_graphs_and_generate_pair_example(SG1, SG2_perturbed_graph))
#
#         for SG1_perturbed_graph in SG1_perturbed_graphs:
#             train_samples_list.append(compare_graphs_and_generate_pair_example(SG1_perturbed_graph, SG2))

In [ ]:
def generate_random_ged_paris(annotated_subgraph: AnnotatedGraph, ged_dist: int, pairs_n: int, is_negative_example: bool):
    train_samples_list = []
    SG1 = annotated_subgraph

    ged_graph_generator = GED_graph_generator(SG1.g, ged_dist)

    # generate pairs

    for i, ged_perturbed_graph in enumerate(ged_graph_generator.generate()):
        if i == pairs_n:
            break

        # positive example
        train_samples_list.append(generate_pair_example(SG1, AnnotatedGraph(ged_perturbed_graph), is_negative_example = is_negative_example))

    return train_samples_list

In [ ]:
import random

random_k_subgraphs = k_subgraph_annotated_graphs.copy()
random.shuffle(random_k_subgraphs)
random_k_subgraphs = random_k_subgraphs#[:10]

pairs_n = -1

total_tasks = len(random_k_subgraphs)
with tqdm_joblib(tqdm(desc="My calculation", total=total_tasks)) as progress_bar:
    ged_2_examples = Parallel(n_jobs=int(cpu_count()), prefer='processes')(
        delayed(generate_random_ged_paris)(annotated_subgraph=annotated_subgraph, ged_dist=2, pairs_n=pairs_n, is_negative_example=True)
        for annotated_subgraph in random_k_subgraphs
    )

with tqdm_joblib(tqdm(desc="My calculation", total=total_tasks)) as progress_bar:
    ged_1_examples = Parallel(n_jobs=int(cpu_count()), prefer='processes')(
        delayed(generate_random_ged_paris)(annotated_subgraph=annotated_subgraph, ged_dist=1, pairs_n=pairs_n, is_negative_example=False)
        for annotated_subgraph in random_k_subgraphs
    )

In [ ]:
# debug_ged_res = generate_random_ged_paris(annotated_subgraph=random_k_subgraphs[0], ged_dist=2, pairs_n=pairs_n, is_negative_example=True)

In [ ]:
ged_examples1 = [elem for lst in ged_1_examples for elem in lst]
ged_examples2 = [elem for lst in ged_2_examples for elem in lst]
ged_examples = ged_examples1 + ged_examples2

In [ ]:
train_samples_list = ged_examples

In [ ]:
len(ged_examples)

In [ ]:
for k_subgraph in k_subgraphs:
    if k_subgraphs[0].nodes == k_subgraph.nodes:
        print("yes")

In [ ]:
len(k_subgraphs)

In [ ]:
len(train_samples_list)

In [ ]:
import wwl
from wwl import WL_metric
from igraph import Graph
from sklearn.metrics.pairwise import laplacian_kernel

def get_graph_wl_distances(graphs, node_label):
    igraphs = [Graph() for i in range(len(graphs))]
    igraphs = [igraph.from_networkx(graph) for igraph, graph in zip(igraphs, graphs)]
    kernel_values = wwl.pairwise_wasserstein_distance(igraphs, [len(igraphs)-1], enforce_continuous=True, num_iterations=10)
    kernel_values = laplacian_kernel(kernel_values) #normalize energey

    # gk_wl = WL_metric.GK_WL()
    # kernel_values = gk_wl.compare_list(graphs, h=5, node_label=node_label)
    return 1- kernel_values
    #return np.round(1 - kernel_values, decimals=4)

def generate_wl_dist_for_ged_pairs(ged_examples_list):
    pairs_index_to_dist_map = {}

    all_ged_example_list_graphs = []
    for ged_example_list in ged_examples_list:
        ged_example_list_graphs = [ged_example_list[0].subgraph.g] + [pair.masked_graph.g for pair in ged_example_list]
        all_ged_example_list_graphs += ged_example_list_graphs

    wl_dist_matrix = get_graph_wl_distances(all_ged_example_list_graphs, node_label=False)

    base_graph_index = 0
    pairs_index = 0
    for ged_example_list_index, ged_example_list in enumerate(ged_examples_list):
        for relative_index, pair in enumerate(ged_example_list):
            wl_dist = wl_dist_matrix[base_graph_index][base_graph_index + relative_index + 1]
            pairs_index_to_dist_map[pairs_index] = wl_dist

            pairs_index += 1
        base_graph_index += 1 + (relative_index + 1)

    return pairs_index_to_dist_map

# go over pairs in ged_examples1/2, and for their subgraphs generate wl+dist
# only take into account their wl_dist from the all pairs vs all pairs matrix

ged_1_examples_pairs_index_to_dist_map = {}
ged_2_examples_pairs_index_to_dist_map = {}

ged_examples_pairs_index_to_dist_map = generate_wl_dist_for_ged_pairs(ged_1_examples + ged_2_examples)
for pair_index, wl_dist in ged_examples_pairs_index_to_dist_map.items():
    if pair_index < len(ged_1_examples):
        ged_1_examples_pairs_index_to_dist_map[pair_index] = wl_dist
    else:
        ged_2_examples_pairs_index_to_dist_map[pair_index] = wl_dist
# ged_1_examples_pairs_index_to_dist_map = generate_wl_dist_for_ged_pairs(ged_1_examples)
# ged_2_examples_pairs_index_to_dist_map = generate_wl_dist_for_ged_pairs(ged_2_examples)

In [ ]:
max(ged_1_examples_pairs_index_to_dist_map.values())

In [ ]:
min(ged_2_examples_pairs_index_to_dist_map.values())

In [ ]:
from scipy.stats import norm
import seaborn as sns
import matplotlib.pyplot as plt

def plot_histogram(sequence, x_title=""):
    fig, ax = plt.subplots(1)
    sns.histplot(sequence, bins='auto', stat='density',
                 label='Normalized Histogram', ax=ax)

    # Fit a distribution to the data
    mu, std = norm.fit(sequence)
    x = np.linspace(min(sequence), max(sequence), 100)
    if not np.all(x == x[0]):
        y = norm.pdf(x, mu, std)
    else:
        y = np.ones_like(x)
    sns.lineplot(x=x, y=y, color='red', label='Fitted Normal Distribution', ax=ax)

    # Plot the KDE
    sns.kdeplot(sequence, label='KDE', bw_adjust=0.5, ax=ax)

    ax.set_xlabel(x_title)
    ax.set_ylabel('Density')
    ax.set_title(f'{x_title} Distribution')
    ax.legend(loc='upper left')
    ax.grid(True)
    plt.show()

In [ ]:
# set(ged_2_examples[0][1].masked_graph.g.edges).symmetric_difference(set(ged_2_examples[0][1].subgraph.g.edges))

In [ ]:
plot_histogram(list(ged_1_examples_pairs_index_to_dist_map.values()), "GED1 WL distance")

In [ ]:
plot_histogram(list(ged_2_examples_pairs_index_to_dist_map.values()), "GED2 WL distance")

check 1-WL distance

In [ ]:
# optimized single call for WL distance calculation
### NOTE: Wl dist is not stable (depends which batch of graph you applied it with) ###
# wl_dist_matrix = get_graph_wl_distances(k_subgraphs, node_label=False)

In [ ]:
# negative_pairs_wl_distances = []
# positive_pairs_wl_distances = []
#
# positive_pairs_index_to_dist_map = {}
# negative_pairs_index_to_dist_map = {}
#
# for i, train_sample in enumerate(train_samples_list):
#     is_negative_sample = train_sample.is_negative_sample
#     graph1 = train_sample.subgraph
#     graph2 = train_sample.masked_graph
#
#     # s2v_graphs = generate_s2v_graphs([graph1.g, graph2.g], print_stats=False)
#
#     # distance = graph_metric_nn.forward([(s2v_graphs[0], s2v_graphs[1])]).item()
#
#     # wl_dist = get_graph_wl_distances([graph1.g, graph2.g], node_label=False)[0][1]
#
#     graph1_index, graph2_index = train_sample_indices_tuple_list[i]
#     wl_dist = wl_dist_matrix[graph1_index][graph2_index]
#     wl_dist = round(wl_dist, 6)
#
#     if is_negative_sample:
#         negative_pairs_wl_distances.append(wl_dist)
#         negative_pairs_index_to_dist_map[i] = wl_dist
#     else:
#         positive_pairs_wl_distances.append(wl_dist)
#         positive_pairs_index_to_dist_map[i] = wl_dist

In [ ]:
# len(positive_pairs_wl_distances)

In [ ]:
# plot_histogram(positive_pairs_wl_distances, "positive WL distance")

In [ ]:
# plot_histogram(negative_pairs_wl_distances, "negative WL distance")

In [ ]:
# cnt = 0
# for i, dist in negative_pairs_index_to_dist_map.items():
#     if dist < 0.13:
#         print(i)

In [ ]:
# from subgraph_matching_via_nn.training.MarginLoss import MarginLoss
# def calc_margin_loss(distances, pos_labels, margin):
#     negative_labels = 1 - pos_labels
#     lossCriterion = MarginLoss(margin)
#     pos_loss, neg_loss = lossCriterion.get_loss(distances, pos_labels, negative_labels)
#     print(pos_loss.sum().item())
#     print(neg_loss.sum().item())
#
# margin = 0.001
# calc_margin_loss(torch.tensor(positive_pairs_wl_distances+negative_pairs_wl_distances), torch.cat((torch.ones(len(positive_pairs_wl_distances)), torch.zeros(len(negative_pairs_wl_distances)))), margin)

Fit GNN on k subgraph pairs

In [ ]:
# disable debugging overhead operations
torch.autograd.set_detect_anomaly(mode=False)

In [ ]:
solver_params['margin_loss_margin_value'] = 0.25 #0.01

In [ ]:
used_train_samples_list = train_samples_list#[0:100]#[:10_000] #Needed due to performance issues
processes_device_ids = [0, 1, 2, 3]

In [ ]:
len(used_train_samples_list)

In [ ]:
val_samples_list = used_train_samples_list
model_factory_func = lambda device: GraphCNN(num_layers=5, num_mlp_layers = 2, input_dim=input_dim, hidden_dim=128, output_dim=num_classes, final_dropout=0.5, learn_eps=False, graph_pooling_type="sum", neighbor_pooling_type="sum", device=device)
solver_params["batch_size"] = 512 * 32 #2048 #1024 #512
solver_params["k_update_plot"] = 1
solver_params["lr"] = 1e-4
solver_params["max_epochs"] = 2000
trainer, graph_metric_nn, model = init_embedding_net_and_trainer(model_factory_func)


# trainer.set_existing_data_loader_paths(
#     ['./dataloaders/1702148493.0283618.p',
#  './dataloaders/1702148497.9922142.p',
#  './dataloaders/1702148502.9499629.p',
#  './dataloaders/1702148511.1819935.p'],
# []
# )

In [ ]:
trainer.train(processes_device_ids=processes_device_ids, use_existing_data_loaders=False, train_samples_list=used_train_samples_list, val_samples_list=[])

Train again without rebuilding data loaders, and without the val_loader

In [ ]:
# model_factory_func = lambda device: GraphCNN(num_layers=5, num_mlp_layers = 2, input_dim=input_dim, hidden_dim=64, output_dim=num_classes, final_dropout=0.5, learn_eps=False, graph_pooling_type="sum", neighbor_pooling_type="sum", device=device)
# new_trainer, graph_metric_nn, model = init_embedding_net_and_trainer(model_factory_func)

# new_trainer.previous_train_loader_paths = trainer.previous_train_loader_paths
# new_trainer.previous_val_loader_paths = [None for device_id in processes_device_ids] #trainer.previous_val_loader_paths
# new_trainer.train(processes_device_ids=processes_device_ids, use_existing_data_loaders=True)

In [ ]:
# model.eval()
# for i in range(0, len(train_samples_list)):
#     is_negative_sample = train_samples_list[i].is_negative_sample
#     graph1 = train_samples_list[i].subgraph
#     graph2 = train_samples_list[i].masked_graph
#
#     s2v_graphs = generate_s2v_graphs([graph1.g, graph2.g], print_stats=False)
#
#     distance = graph_metric_nn.forward([(s2v_graphs[0], s2v_graphs[1])]).item()
#
#     if is_negative_sample and (distance < solver_params['margin_loss_margin_value']):
#         print(f"negative loss term not zero, as distance for this negative example pair is {distance}")
#     if (not is_negative_sample) and (distance > solver_params['margin_loss_margin_value']):
#         print(f"negative loss term not zero, as distance for this positive example pair is {distance}")

Execution time profiling

In [ ]:
# from cProfile import Profile
# from pstats import SortKey, Stats

# with Profile() as profile:
#     new_trainer.train(processes_device_ids=processes_device_ids, use_existing_data_loaders=True)
#     (
#     Stats(profile)
#         .strip_dirs()
#         .sort_stats(SortKey.CUMULATIVE)
#         .print_stats()
#     )